In [60]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = dfs[0]

In [3]:
df.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

In [4]:
df.Borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [5]:
df = df[df['Borough']!='Not assigned']
df[df['Borough']=='Not assigned']

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []

In [6]:
df.head()

Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [7]:
df.loc[df['Neighbourhood'] == 'Not assigned'] # test

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []

I have cleaned the dataframe based on the instruction !

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
df.shape

(103, 3)

In [9]:
geo = pd.read_csv('C:\\Projects\\IBM_Final_capstone\\Geospatial_Coordinates.csv')

In [10]:
geo.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [11]:
df=df.merge(geo,how='inner',on='Postal Code')

In [12]:
df.head()

Postal Code           Borough                                Neighbourhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

Use the K-Means clustering to get new insights !

In [17]:
CLIENT_ID = 'OIOONDYWS2FMPWFARPS5C5TLC231UJLL1PMYRWZGFKTZIE1R' # your Foursquare ID
CLIENT_SECRET = '3YDWUYZJRSSFMHQBMWGRDUUMVGBCWZSTNTRCMEOQ35HB1SW5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OIOONDYWS2FMPWFARPS5C5TLC231UJLL1PMYRWZGFKTZIE1R
CLIENT_SECRET:3YDWUYZJRSSFMHQBMWGRDUUMVGBCWZSTNTRCMEOQ35HB1SW5


In [18]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [29]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API


radius = 1000 # define radius



# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OIOONDYWS2FMPWFARPS5C5TLC231UJLL1PMYRWZGFKTZIE1R&client_secret=3YDWUYZJRSSFMHQBMWGRDUUMVGBCWZSTNTRCMEOQ35HB1SW5&v=20180605&ll=43.7532586,-79.3296565&radius=1000&limit=1000'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fcbf0d903267f4682055e61'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 28,
  'suggestedBounds': {'ne': {'lat': 43.762258609000014,
    'lng': -79.31721997969855},
   'sw': {'lat': 43.74425859099999, 'lng': -79.34209302030145}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display'

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-32-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


name            categories        lat        lng
0    Allwyn's Bakery  Caribbean Restaurant  43.759840 -79.324719
1    Brookbanks Park                  Park  43.751976 -79.332140
2        Tim Hortons                  Café  43.760668 -79.326368
3                A&W  Fast Food Restaurant  43.760643 -79.326865
4  Bruno's valu-mart         Grocery Store  43.746143 -79.324630

In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

28 venues were returned by Foursquare.


In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(4882, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Parkwoods              43.753259              -79.329656   
1    Parkwoods              43.753259              -79.329656   
2    Parkwoods              43.753259              -79.329656   
3    Parkwoods              43.753259              -79.329656   
4    Parkwoods              43.753259              -79.329656   

               Venue  Venue Latitude  Venue Longitude        Venue Category  
0    Allwyn's Bakery       43.759840       -79.324719  Caribbean Restaurant  
1    Brookbanks Park       43.751976       -79.332140                  Park  
2        Tim Hortons       43.760668       -79.326368                  Café  
3                A&W       43.760643       -79.326865  Fast Food Restaurant  
4  Bruno's valu-mart       43.746143       -79.324630         Grocery Store

In [39]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                             
Agincourt                                                           51   
Alderwood, Long Branch                                              24   
Bathurst Manor, Wilson Heights, Downsview North                     29   
Bayview Village                                                     14   
Bedford Park, Lawrence Manor East                                   40   
...                                                                ...   
Willowdale, Willowdale West                                         11   
Woburn                                                               8   
Woodbine Heights                                                    27   
York Mills West                                                     20   
York Mills, Silver Hills                                             4   

                                                 Neighborhood Longitude  \
Neighborhood                                                              
Agincourt                                                            51   
Alderwood, Long Branch                                               24   
Bathurst Manor, Wilson Heights, Downsview North                      29   
Bayview Village                                                      14   
Bedford Park, Lawrence Manor East                                    40   
...                                                                 ...   
Willowdale, Willowdale West                                          11   
Woburn                                                                8   
Woodbine Heights                                                     27   
York Mills West                                                      20   
York Mills, Silver Hills                                              4   

                                                 Venue  Venue Latitude  \
Neighborhood                                                             
Agincourt                                           51              51   
Alderwood, Long Branch                              24              24   
Bathurst Manor, Wilson Heights, Downsview North     29              29   
Bayview Village                                     14              14   
Bedford Park, Lawrence Manor East                   40              40   
...                                                ...             ...   
Willowdale, Willowdale West                         11              11   
Woburn                                               8               8   
Woodbine Heights                                    27              27   
York Mills West                                     20              20   
York Mills, Silver Hills                             4               4   

                                                 Venue Longitude  \
Neighborhood                                                       
Agincourt                                                     51   
Alderwood, Long Branch                                        24   
Bathurst Manor, Wilson Heights, Downsview North               29   
Bayview Village                                               14   
Bedford Park, Lawrence Manor East                             40   
...                                                          ...   
Willowdale, Willowdale West                                   11   
Woburn                                                         8   
Woodbine Heights                                              27   
York Mills West                                               20   
York Mills, Silver Hills                                       4   

                                                 Venue Category  
Neighborhood                                                     
Agincourt                                                    51  
Alderwood, Long Branch                                       24 

In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 334 uniques categories.


In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Zoo  Accessories Store  Afghan Restaurant  African Restaurant  Airport  \
0    0                  0                  0                   0        0   
1    0                  0                  0                   0        0   
2    0                  0                  0                   0        0   
3    0                  0                  0                   0        0   
4    0                  0                  0                   0        0   

   American Restaurant  Amphitheater  Animal Shelter  Antique Shop  Aquarium  \
0                    0             0               0             0         0   
1                    0             0               0             0         0   
2                    0             0               0             0         0   
3                    0             0               0             0         0   
4                    0             0               0             0         0   

   ...  Video Game Store  Video Store  Vietnamese Restaurant  Warehouse Store  \
0  ...                 0            0                      0                0   
1  ...                 0            0                      0                0   
2  ...                 0            0                      0                0   
3  ...                 0            0                      0                0   
4  ...                 0            0                      0                0   

   Whisky Bar  Wine Bar  Wine Shop  Wings Joint  Women's Store  Yoga Studio  
0           0         0          0            0              0            0  
1           0         0          0            0              0            0  
2           0         0          0            0              0            0  
3           0         0          0            0              0            0  
4           0         0          0            0              0            0  

[5 rows x 334 columns]

In [43]:
toronto_onehot.shape

(4882, 334)

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Zoo  Accessories Store  \
0                                         Agincourt  0.0                0.0   
1                            Alderwood, Long Branch  0.0                0.0   
2   Bathurst Manor, Wilson Heights, Downsview North  0.0                0.0   
3                                   Bayview Village  0.0                0.0   
4                 Bedford Park, Lawrence Manor East  0.0                0.0   
..                                              ...  ...                ...   
93                      Willowdale, Willowdale West  0.0                0.0   
94                                           Woburn  0.0                0.0   
95                                 Woodbine Heights  0.0                0.0   
96                                  York Mills West  0.0                0.0   
97                         York Mills, Silver Hills  0.0                0.0   

    Afghan Restaurant  African Restaurant  Airport  American Restaurant  \
0                 0.0                 0.0      0.0                0.000   
1                 0.0                 0.0      0.0                0.000   
2                 0.0                 0.0      0.0                0.000   
3                 0.0                 0.0      0.0                0.000   
4                 0.0                 0.0      0.0                0.025   
..                ...                 ...      ...                  ...   
93                0.0                 0.0      0.0                0.000   
94                0.0                 0.0      0.0                0.000   
95                0.0                 0.0      0.0                0.000   
96                0.0                 0.0      0.0                0.000   
97                0.0                 0.0      0.0                0.000   

    Amphitheater  Animal Shelter  Antique Shop  ...  Video Game Store  \
0            0.0             0.0           0.0  ...               0.0   
1            0.0             0.0           0.0  ...               0.0   
2            0.0             0.0           0.0  ...               0.0   
3            0.0             0.0           0.0  ...               0.0   
4            0.0             0.0           0.0  ...               0.0   
..           ...             ...           ...  ...               ...   
93           0.0             0.0           0.0  ...               0.0   
94           0.0             0.0           0.0  ...               0.0   
95           0.0             0.0           0.0  ...               0.0   
96           0.0             0.0           0.0  ...               0.0   
97           0.0             0.0           0.0  ...               0.0   

    Video Store  Vietnamese Restaurant  Warehouse Store  Whisky Bar  Wine Bar  \
0         0.000               0.019608              0.0         0.0       0.0   
1         0.000               0.000000              0.0         0.0       0.0   
2         0.000               0.000000              0.0         0.0       0.0   
3         0.000               0.000000              0.0         0.0       0.0   
4         0.025               0.000000              0.0         0.0       0.0   
..          ...                    ...              ...         ...       ...   
93        0.000               0.000000              0.0         0.0       0.0   
94        0.000               0.000000              0.0         0.0       0.0   
95        0.000               0.000000              0.0         0.0       0.0   
96        0.000               0.000000              0.0         0.0       0.0   
97        0.000               0.000000              0.0         0.0       0.0   

    Wine Shop  Wings Joint  Women's Store  Yoga Studio  
0         0.0        0.000            0.0          0.0  
1         0.0        0.000            0.0          0.0  
2         0.0        0.000            0.0          0.0  
3         0.0        0.000            0.0          0.0  
4         0.0        0.025            0.0          0.0

In [45]:
toronto_grouped.shape

(98, 334)

In [46]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.14
1         Shopping Mall  0.06
2  Caribbean Restaurant  0.04
3                Bakery  0.04
4        Sandwich Place  0.04


----Alderwood, Long Branch----
               venue  freq
0     Discount Store  0.12
1  Convenience Store  0.08
2        Pizza Place  0.08
3           Pharmacy  0.08
4       Intersection  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
                      venue  freq
0                      Bank  0.07
1               Coffee Shop  0.07
2                Restaurant  0.03
3          Sushi Restaurant  0.03
4  Mediterranean Restaurant  0.03


----Bayview Village----
                 venue  freq
0                 Bank  0.14
1  Japanese Restaurant  0.14
2        Grocery Store  0.14
3          Gas Station  0.14
4         Skating Rink  0.07


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2                

                venue  freq
0         Coffee Shop  0.10
1    Sushi Restaurant  0.06
2     Thai Restaurant  0.05
3  Italian Restaurant  0.05
4                Park  0.04


----The Annex, North Midtown, Yorkville----
                           venue  freq
0                           Café  0.05
1             Italian Restaurant  0.05
2                            Gym  0.04
3                     Restaurant  0.04
4  Vegetarian / Vegan Restaurant  0.04


----The Beaches----
                 venue  freq
0                  Pub  0.08
1          Coffee Shop  0.07
2          Pizza Place  0.05
3  Japanese Restaurant  0.04
4       Breakfast Spot  0.04


----The Danforth West, Riverdale----
                venue  freq
0    Greek Restaurant  0.10
1         Coffee Shop  0.07
2                Café  0.06
3                 Pub  0.05
4  Italian Restaurant  0.03


----The Kingsway, Montgomery Road, Old Mill North----
                venue  freq
0         Coffee Shop  0.09
1    Sushi Restaurant  0.04
2        

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Agincourt    Chinese Restaurant   
1                           Alderwood, Long Branch        Discount Store   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Bank   
4                Bedford Park, Lawrence Manor East    Italian Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0         Shopping Mall        Sandwich Place           Coffee Shop   
1     Convenience Store           Pizza Place              Pharmacy   
2                  Bank      Community Center              Ski Area   
3           Gas Station         Grocery Store   Japanese Restaurant   
4           Coffee Shop           Pizza Place                  Bank   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0  Caribbean Restaurant                Bakery     Indian Restaurant   
1                   Pub                 Trail         Shopping Mall   
2      Sushi Restaurant           Supermarket    Frozen Yogurt Shop   
3                  Park            Restaurant                  Café   
4        Sandwich Place     Indian Restaurant          Skating Rink   

  8th Most Common Venue     9th Most Common Venue 10th Most Common Venue  
0          Skating Rink            Clothing Store       Sushi Restaurant  
1         Garden Center               Gas Station             Donut Shop  
2   Fried Chicken Joint  Mediterranean Restaurant            Bridal Shop  
3          Skating Rink        Chinese Restaurant                  Trail  
4                Bakery          Sushi Restaurant           Liquor Store

# Cluster Neighborhoods

In [53]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 2, 2, 0, 2, 2, 2])

In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels2', kmeans.labels_)

toronto_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

Postal Code           Borough                                Neighbourhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster Labels2  Cluster Labels  \
0  43.753259 -79.329656              3.0             3.0   
1  43.725882 -79.315572              3.0             3.0   
2  43.654260 -79.360636              2.0             2.0   
3  43.718518 -79.464763              2.0             2.0   
4  43.662301 -79.389494              2.0             2.0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                  Park     Convenience Store         Shopping Mall   
1           Coffee Shop  Gym / Fitness Center           Men's Store   
2           Coffee Shop                   Pub                  Park   
3        Clothing Store  Fast Food Restaurant            Restaurant   
4           Coffee Shop                  Park      Sushi Restaurant   

  4th Most Common Venue 5th Most Common Venue  6th Most Common Venue  \
0              Bus Stop              Pharmacy           Tennis Court   
1         Grocery Store     French Restaurant             Boxing Gym   
2               Theater                  Café             Restaurant   
3           Coffee Shop      Sushi Restaurant  Vietnamese Restaurant   
4    Italian Restaurant   Japanese Restaurant        Thai Restaurant   

  7th Most Common Venue 8th Most Common Venue      9th Most Common Venue  \
0                  Café        Shop & Service         Chinese Restaurant   
1           Pizza Place            Playground        Sporting Goods Shop   
2                Bakery        Breakfast Spot                      Diner   
3          Dessert Shop   Fried Chicken Joint     Furniture / Home Store   
4                  Café           Men's Store  Middle Eastern Restaurant   

  10th Most Common Venue  
0            Supermarket  
1           Intersection  
2  Performing Arts Venue  
3               Boutique  
4              Gastropub

In [81]:
toronto_merged.dropna(axis=0,inplace=True)

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters